<a href="https://colab.research.google.com/github/lara-sj/cots-detection-model/blob/main/GBR_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import cv2
import ast
import glob
import torch
import importlib
import traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

import shutil
import sys
#sys.path.append('../input/tensortflow-great-barrier-reef')

from PIL import Image
from IPython.display import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
TRAIN_IMAGES = '/content/drive/MyDrive/great_barrier_reef/train/images'
TRAIN_LABELS = '/content/drive/MyDrive/great_barrier_reef/train/labels'

VALID_IMAGES = '/content/drive/MyDrive/great_barrier_reef/valid/images'
VALID_LABELS = '/content/drive/MyDrive/great_barrier_reef/valid/labels'

In [ ]:
%cd /content/drive/MyDrive/great_barrier_reef
#!git clone https://github.com/ultralytics/yolov5.git

/content/drive/MyDrive/great_barrier_reef


In [ ]:
%cd yolov5

/content/drive/MyDrive/great_barrier_reef/yolov5


In [ ]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3) #####learning rate
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
anchors: 0  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction) #####aug
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction) #####aug
hsv_v: 0.4  # image HSV-Value augmentation (fraction) #####aug
degrees: 0.0  # image rotation (+/- deg).  #####aug
translate: 0.1  # image translation (+/- fraction).  #####aug
scale: 0.5  # image scale (+/- gain). #####aug
shear: 0.0  # image shear (+/- deg).  #####aug
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 #####aug
flipud: 0.5  # image flip up-down (probability) #####aug
fliplr: 0.5  # image flip left-right (probability) #####aug
mosaic: 1.0  # image mosaic (probability) #####aug
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [ ]:
data = '''

train: /content/drive/MyDrive/great_barrier_reef/train/images  # train images
val: /content/drive/MyDrive/great_barrier_reef/valid/images  # val images 
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['cots']  # class names

'''

In [ ]:
with open('/content/drive/MyDrive/great_barrier_reef/yolov5/data/gbr_data.yaml', 'w') as fp:
    fp.write(data)
with open('/content/drive/MyDrive/great_barrier_reef/yolov5/data/hyps/gbr_hyps.yaml', 'w') as fp:
    fp.write(hyps)

In [ ]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 7.5 MB/s 
     |████████████████████████████████| 181 kB 67.0 MB/s 
     |████████████████████████████████| 144 kB 68.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="great-barrier-reef", entity="lara-sj")

wandb: Currently logged in as: lara-sj (use `wandb login --relogin` to force relogin)


In [ ]:
FOLD = 1
DIM = 1440
MODEL = 'yolov5s'
BATCH = 16
EPOCHS = 300
OPTIMIZER = 'Adam'
NAME = f'{MODEL}-dim{DIM}-fold{FOLD}'
PROJECT   = 'great-barrier-reef'

In [ ]:
!pip install PyYAML==5.4.1
!python train.py --weights {MODEL} --img {DIM} --batch {BATCH} --epochs {EPOCHS} --cfg yolov5s.yaml --optimizer {OPTIMIZER} --data /content/drive/MyDrive/great_barrier_reef/yolov5/data/gbr_data.yaml --hyp /content/drive/MyDrive/great_barrier_reef/yolov5/data/hyps/gbr_hyps.yaml

     |████████████████████████████████| 636 kB 8.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


wandb: Currently logged in as: lara-sj (use `wandb login --relogin` to force relogin)
train: weights=yolov5s, cfg=yolov5s.yaml, data=/content/drive/MyDrive/great_barrier_reef/yolov5/data/gbr_data.yaml, hyp=/content/drive/MyDrive/great_barrier_reef/yolov5/data/hyps/gbr_hyps.yaml, epochs=300, batch_size=16, imgsz=1440, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0

In [ ]:
!pip install PyYAML==5.4.1
!python train.py --resume

     |████████████████████████████████| 636 kB 15.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
train: weights=yolov5s.pt, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
Resuming training from ./runs/train/exp31/weights/last.pt
YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281M

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!python detect.py --source /content/drive/MyDrive/great_barrier_reef/valid/images --weights /content/drive/MyDrive/great_barrier_reef/yolov5/runs/train/exp17/weights/best.pt --data /content/drive/MyDrive/great_barrier_reef/yolov5/data/gbr_data.yaml --img 1280 --save-txt --save-conf